In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
import seaborn as sb
df = pd.read_csv('data.csv')
df
df.info()

df.describe().T

df.duplicated().sum()
df.shape

df.isna().sum()
df.dropna()
df.max()
df.columns

# Calculate the 33rd and 66th percentiles
low_threshold = df['AveragScore'].quantile(0.33)
high_threshold = df['AveragScore'].quantile(0.66)

# Function to classify Prosperity Index into Low, Medium, High
def classify_prosperity(index):
    if index < low_threshold:
        return 0
    elif index < high_threshold:
        return 1
    else:
        return 2

# Apply classification
df['ProsperityClass'] = df['AveragScore'].apply(classify_prosperity)
df
from sklearn.utils import shuffle

df = shuffle(df).reset_index(drop=True)
df

df['Country'] = df['Country'].str.strip()

numeric_df = df.drop(columns=['Country', 'ProsperityClass'])

numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')

plt.figure(figsize=(14, 10))
corr_matrix = numeric_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Development Indicators', fontsize=16)
plt.show()
from sklearn.ensemble import RandomForestClassifier

# Features and target variable
X = df.drop(['Country', 'ProsperityClass', 'AveragScore'], axis=1)  # Features
y = df['ProsperityClass']  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importance
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances}).sort_values(by='Importance')

# Output feature importance
print(feature_importance_df)

import seaborn as sns
import matplotlib.pyplot as plt

# Create the scatter plot (no color added to the points)
sns.scatterplot(x='Governance', y='LivingConditions', data=df)

# Customize the plot title and labels
plt.title('The relationship between quality of governance and quality of life')

# Change the colors of the axes
plt.gca().spines['bottom'].set_color('red')   # Color x-axis in red
plt.gca().spines['left'].set_color('blue')    # Color y-axis in blue

# Set axis labels and colors
plt.xlabel('Quality of governance', color='red')   # Red x-axis label
plt.ylabel('Quality of life', color='blue')        # Blue y-axis label

# Make axes thicker for visibility
plt.gca().spines['bottom'].set_linewidth(2)   # Thicker x-axis
plt.gca().spines['left'].set_linewidth(2)     # Thicker y-axis

# Show the plot
plt.show()

sns.scatterplot(x='Governance', y='Health', data=df)
plt.title('The relationship between governance quality and health')
plt.xlabel('Quality of governance')
plt.ylabel('Health')
plt.show()
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Education', y='PersonelFreedom')
plt.title('The relationship between level of education and personal freedom')
plt.xlabel('Education level')
plt.ylabel('Personal freedom')
plt.show()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=df, x='LivingConditions', y='Education')
plt.title('Relationship between Living Conditions and Education')
plt.xlabel('Living Conditions')
plt.ylabel('Education')
plt.show()
import seaborn as sns
import matplotlib.pyplot as plt

indicators = ['SafetySecurity', 'PersonelFreedom', 'Governance', 'SocialCapital', 
              'InvestmentEnvironment', 'EnterpriseConditions', 'MarketAccessInfrastructure', 
              'EconomicQuality', 'LivingConditions', 'Health', 'Education', 'NaturalEnvironment']

plt.figure(figsize=(18, 10))
sns.boxplot(data=df[indicators], palette='Set3')
plt.title('Boxplot of Development Indicators Across All Countries', fontsize=18, fontweight='bold')
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Indicator Value', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()
df.hist(figsize=(14,14))
plt.show()
 df['ProsperityClass'].value_counts()
df
#frequency of target using bar chart
plt.bar(df['ProsperityClass'].unique(), df['ProsperityClass'].value_counts(), color = ['green', 'yellow','red'])

plt.xlabel('Target Classes : yellow(medium), red(low), green(high)')
plt.ylabel('Count')
plt.title('Count of each Target Class')
import joblib
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical features

df_scaled =pd.DataFrame(scaler.fit_transform(df.drop(columns=['Country','Health', 'EconomicQuality','AveragScore' ,'ProsperityClass','SocialCapital', 'NaturalEnvironment','EnterpriseConditions', 'PersonelFreedom', 'SafetySecurity'],axis = 1)))
scaled_df = pd.concat([df_scaled,df['ProsperityClass']],axis = 1)

# Save the scaler and the trained model
joblib.dump(scaler, 'scaler.pkl')


scaled_df
df = scaled_df
df
# Features and target variable
X = df.drop('ProsperityClass', axis=1)  # Features
y = df['ProsperityClass']  # Target variable
X.columns = X.columns.astype(str)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier

rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)
joblib.dump(rf, 'random_forest_model.pkl')
# Evaluate Decision Tree Model
y_pred = rf.predict(X_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict the target variable for the test set
y_pred = rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for multiclass
recall = recall_score(y_test, y_pred, average='weighted')        # Weighted for multiclass
f1 = f1_score(y_test, y_pred, average='weighted')                # Weighted for multiclass

# Print the metrics
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print(f"Model Precision: {precision * 100:.2f}%")
print(f"Model Recall: {recall * 100:.2f}%")
print(f"Model F1-Score: {f1 * 100:.2f}%")

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict the target variable for the test set
y_pred = rf.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Metrics as a dictionary
metrics = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1-Score": f1
}

# Plotting
plt.figure(figsize=(8, 5))
plt.bar(metrics.keys(), metrics.values(), color=['blue', 'green', 'orange', 'red'])
plt.title('Model Performance Metrics', fontsize=14)
plt.ylabel('Score', fontsize=12)
plt.ylim(0, 1)  # Scores range from 0 to 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

import matplotlib.pyplot as plt
import numpy as np

# Get feature importance
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]  # Sort in descending order
feature_names = X.columns

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.title("Feature Importance")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

accuracies = []
n_estimators = [10, 20, 50, 100, 200]

for n in n_estimators:
    rf_temp = RandomForestClassifier(n_estimators=n, random_state=42)
    rf_temp.fit(X_train, y_train)
    y_pred_temp = rf_temp.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred_temp))

# Plot accuracy vs. number of trees
plt.figure(figsize=(8, 6))
plt.plot(n_estimators, accuracies, marker='o')
plt.title('Accuracy vs. Number of Trees')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Accuracy')
plt.show()

# Scatter plot of actual vs. predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'k--', lw=3)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted Values')
plt.show()

# Pair plot to visualize relationships between features and prosperity class
sns.pairplot(df, hue='ProsperityClass', palette='Set1')
plt.show()

import shap

# Use the TreeExplainer from the SHAP library
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values[1], X_test)

import joblib

# Save the trained model to a file
joblib.dump(rf, 'random_forest_model.pkl')

import pickle

# Save the trained model to a file
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

import streamlit as st
import pandas as pd
import joblib
import numpy as np

# Load the saved Random Forest model and scaler
model = joblib.load('random_forest_model.pkl')
scaler = joblib.load('scaler.pkl')

# Define the feature names
feature_names = [
    'Governance', 'LivingConditions', 'Health', 'Education',
    'InvestmentEnvironment', 'MarketAccessInfrastructure', 'EconomicQuality'
]

# Create a function to make predictions
def predict_prosperity_class(input_data):
    # Reshape input data for scaling
    input_data = np.array(input_data).reshape(1, -1)
    
    # Scale the input data using the loaded scaler
    scaled_data = scaler.transform(input_data)
    
    # Make the prediction
    prediction = model.predict(scaled_data)
    
    return prediction[0]

# Title of the Streamlit app
st.title('Prosperity Class Prediction App')

# Create input sliders for each feature
input_values = []
for feature in feature_names:
    value = st.slider(f'Enter value for {feature}', min_value=0.0, max_value=100.0, value=50.0)
    input_values.append(value)

# Button to make predictions
if st.button('Predict Prosperity Class'):
    # Perform the prediction
    prosperity_class = predict_prosperity_class(input_values)
    
    # Display the prediction result
    if prosperity_class == 0:
        st.success('Prediction: Low Prosperity')
    elif prosperity_class == 1:
        st.info('Prediction: Medium Prosperity')
    else:
        st.warning('Prediction: High Prosperity')

# Display feature importance if the user clicks the button
if st.checkbox('Show Feature Importance'):
    importances = model.feature_importances_
    st.bar_chart(pd.DataFrame({'Feature': feature_names, 'Importance': importances}).set_index('Feature'))

# Add an explanation section about the model
st.sidebar.header('Model Details')
st.sidebar.write('This application uses a pre-trained Random Forest model to classify countries into one of three prosperity classes: Low, Medium, or High, based on various development indicators.')


